## Import necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from math import sqrt
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.transforms import Compose, Resize, Grayscale, ToTensor
from torch.nn import Sequential, Conv2d, BatchNorm2d, ReLU, MaxPool2d, functional

from qiskit import Aer, QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import BackendEstimator, BackendSampler
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.neural_networks import EstimatorQNN, SamplerQNN
from qiskit_machine_learning.connectors import TorchConnector

## Load data

Define constants and preprocessing function

In [ ]:
# Constants
IMG_SIZE = (160, 160)  # Image size for resizing
NUM_CLASSES = 4  # Number of classes in the dataset
BATCH_SIZE = 32  # Batch size for data loading

# Preprocessing function (to be used only with QNN)
def scale_image(image):
    """Scale the pixel values so that they are in [0, pi]"""
    image = np.array(image.squeeze())
    scaler = StandardScaler()
    scaled_img= scaler.fit_transform(image)
    scaled_img = scaled_img/max(scaled_img.flatten()) *np.pi/2 + np.pi/2
    
    return torch.tensor(scaled_img).unsqueeze(dim=0)

Load train dataset

In [ ]:
# Load train dataset from class-specific folders
train_ds = ImageFolder(
    root=r"C:\Users\giuseppe.dambruoso\OneDrive - LUTECH SPA\Desktop\Progetto\Dataset\train",
    # Compose a sequence of transformations to apply to each image
    transform=Compose([
        # Resize the image to IMG_SIZE
        Resize(IMG_SIZE),
        # Convert the image to grayscale (single channel)
        Grayscale(num_output_channels=1),
        # Convert the image to a PyTorch tensor
        ToTensor(),
        # Rescale pixel values so that they are between 0 and pi
        lambda x: scale_image(x)
    ]),
    # Compose a sequence of transformations to apply to each target label
    target_transform=Compose([
        # Convert the target labels to a PyTorch tensor
        lambda x: torch.tensor(x),
        # One-hot encode the target labels 
        lambda x: torch.eye(NUM_CLASSES)[x].to(torch.float64)
    ])
)

# Load dataset in batches
train_loader = DataLoader(
    train_ds,  # Training dataset
    batch_size=BATCH_SIZE,  # Batch size for loading data
    shuffle=True,  # Shuffle the data at every epoch
    drop_last=True  # Drop the last batch if it is incomplete
)

Show some images from the train dataset

In [ ]:
figure_width = 15
figure_height = 3
num_images_to_display = 7

for batch_idx, (data, targets) in enumerate(train_loader):  
    plt.figure(figsize=(figure_width, figure_height))
    
    for i in range(num_images_to_display):
        plt.subplot(1, num_images_to_display, i + 1)
        image = np.squeeze(data[i].numpy())
        plt.imshow(image, cmap='gray')
        plt.title(f"Class: {torch.argmax(targets[i]).item()}")
        plt.axis('off')
    plt.suptitle("SOME IMAGES FROM THE TRAIN DATASET", fontsize=14)
    plt.show()
    break

Load validation dataset

In [ ]:
# Load dataset from class-specific folders
validation_ds = ImageFolder(
    root=r"C:\Users\giuseppe.dambruoso\OneDrive - LUTECH SPA\Desktop\Progetto\Dataset\test",  # Path to the validation data directory
    # Compose a sequence of transformations to apply to each image
    transform=Compose([
        # Resize the image to IMG_SIZE
        Resize(IMG_SIZE),
        # Convert the image to grayscale (single channel)
        Grayscale(num_output_channels=1),
        # Convert the image to a PyTorch tensor
        ToTensor(),
        # Rescale pixel values so that they are between 0 and pi
        lambda x: scale_image(x)
    ]),
    # Compose a sequence of transformations to apply to each target
    target_transform=Compose([
        # Convert the target to a PyTorch tensor
        lambda x: torch.tensor(x),
        # One-hot encode the target labels
        lambda x: torch.eye(NUM_CLASSES)[x].to(torch.float64)
    ])
)

# Load dataset in batches
validation_loader = DataLoader(
    validation_ds,  # Validation dataset
    batch_size=BATCH_SIZE,  # Batch size for loading data
    shuffle=True,  # Shuffle the data at every epoch
    drop_last=True  # Drop the last batch if it is incomplete
)

Show some images from the validation dataset

In [ ]:
for batch_idx, (data, targets) in enumerate(validation_loader):  
    plt.figure(figsize=(figure_width, figure_height))
    
    for i in range(num_images_to_display):
        plt.subplot(1, num_images_to_display, i + 1)
        image = np.squeeze(data[i].numpy())
        plt.imshow(image, cmap='gray')
        plt.title(f"Class: {torch.argmax(targets[i]).item()}")
        plt.axis('off')
    plt.suptitle("SOME IMAGES FROM THE VALIDATION DATASET", fontsize=14)
    plt.show()
    break

Show the distribution of the images among the classes

In [ ]:
# Function to calculate class distribution
def calculate_class_distribution(loader):
    class_counts = [0] * NUM_CLASSES
    total_samples = 0
    
    for _, labels in loader:
        for label in labels:
            class_counts[label.argmax().item()] += 1
            total_samples += 1
            
    class_distribution = [count / total_samples for count in class_counts]
    
    return class_distribution

# Calculate class distribution for training dataset
train_class_distribution = calculate_class_distribution(train_loader)

# Calculate class distribution for validation dataset
validation_class_distribution = calculate_class_distribution(validation_loader)

# Class labels
class_labels = ['Class 1', 'Class 2', 'Class 3', 'Class 4']

# Plot pie chart for training dataset
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.pie(train_class_distribution, labels=class_labels, autopct='%1.1f%%')
plt.title('Training Dataset')

# Plot pie chart for validation dataset
plt.subplot(1, 2, 2)
plt.pie(validation_class_distribution, labels=class_labels, autopct='%1.1f%%')
plt.title('Validation Dataset')

plt.show()

## Define the convolutional neural network architecture

#### Define the convolutional block

In [ ]:
def ConvBlock(in_channels, out_channels):
    """
    Defines a convolutional block.

    Args:
        in_channels : The number of input channels.
        out_channels : The number of output channels.

    Returns:
        A sequential module containing convolutional, ReLU, batch 
        normalization and max pooling layers.
    """
    model_cb = torch.nn.Sequential(
        torch.nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=5,
            stride=1,
            padding='same'
        ),
        torch.nn.ReLU(),
        torch.nn.BatchNorm2d(out_channels),
        torch.nn.MaxPool2d(kernel_size=2)
    )
    return model_cb

#### Define the dense block

In [ ]:
def DenseBlock(in_features, out_features):
    """
    Defines a convolutional block.

    Args:
        in_features : The number of input features.
        out_features : The number of output features.

    Returns:
        A sequential module containing linear, ReLU, batch normalization
        and dropout layers.
    """
    model_db = torch.nn.Sequential(
        torch.nn.Linear(
            in_features,
            out_features
        ),
        torch.nn.ReLU(),
        torch.nn.BatchNorm1d(out_features),
        torch.nn.Dropout(p=0.2)
    )
    return model_db

#### Define the quantum convolutional layer

Define the parametrized quantum circuit used in the quantum filter

In [ ]:
# Define the number of qubits that will compose the circuit
N_QUBITS = 4

# Create the feature map and ansatz circuits
feature_map = ZZFeatureMap(N_QUBITS, parameter_prefix='x')
ansatz = RealAmplitudes(N_QUBITS, reps=N_QUBITS, entanglement='linear')

# Create the quantum circuit
pqc = QuantumCircuit(N_QUBITS)

# Add feature map and ansatz to the quantum circuit
pqc.compose(feature_map, inplace=True)
pqc.compose(ansatz, inplace=True)

# Draw the quantum circuit
pqc.draw(output='mpl', style='clifford')

Define the quantum filter used in the quantum convolution

In [ ]:
def quantum_filter(input, pqc, backend, shots):
    """
    Applies a quantum filter to a given 2x2 window of the input image.
    The values of the window become the input parameters of a 4-qubit
    quantum circuit. The quantum circuit is then executed shots times
    on the specified backend. Finally, the average measurement outcome
    is returned as output.

    The execution of the circuit is performed by means of EstimatorQNN,
    which takes also care of all the backpropagation procedures like
    the computation of gradients as well as the update of the trainable
    parameters of the circuit.

    The compatibility between EstimatorQNN and PyTorch is handled by a
    TorchConnector.

    Args:
        input: The window to which the quantum filter will be applied.
        pqc: The quantum circuit associated to the quantum filter.
        backend: The device or simulator that executes the circuit.
        shots: The number of times the circuit will be executed.
    
    Returns:
        The average result of the quantum circuit execution.
    """
    qc_executer = EstimatorQNN(
                    # Set quantum circuit
                    circuit=pqc.decompose(),
                    # Set the estimator that will execute the circuit
                    estimator=BackendEstimator(backend),
                    # Set non-trainable inpu parameters
                    input_params=feature_map.parameters,
                     # Set trainable parameters
                    weight_params=ansatz.parameters,
                    # Compute gradients with respect to input data for a
                    # proper gradient computation with TorchConnector.
                    input_gradients=True
                )

    # Set number of shots for the quantum circuit execution
    qc_executer.estimator.set_options(shots=shots)

    # Make the qc_executer compatible with PyTorch
    quantum_filter = TorchConnector(qc_executer)
    
    # Apply the quantum filter to the flattened input window
    output = quantum_filter(input.reshape(-1)).item()
    return output

Define the quantum convolution

In [ ]:
def quantum_convolution(data_loader, pqc, backend, shots):
   """
    Applies quantum convolution to each image of a specified data loader. 
    It operates by applying a quantum filter to every 2x2 sliding block
    extracted from the image.

    Args:
        data_loader: The data loader containing the images.
        pqc: The quantum circuit associated with the quantum filter.
        backend: The backend associated with the quantum filter.
        shots: The number of shots associated with the quantum filter.
    
    Returns:
        The set of the convoluted images.
   """
   # Unfold each image of tha data loader into 2x2 blocks
   input_unfolded = torch.nn.functional.unfold(
      data_loader, kernel_size=2, stride=2, padding=0).permute(0, 2, 1)

    # Apply the quantum filter to each 2x2 block
   output_unfolded = [[] for _ in range(input_unfolded.size(0))]   
   for i in range(input_unfolded.size(0)):
      for j in range(input_unfolded.size(1)):
         output_unfolded[i].append(
            quantum_filter(input_unfolded[i, j], pqc, backend, shots))
   output_unfolded = torch.tensor(output_unfolded)
    
    # Refold the output images
   output = output_unfolded.view(
      output_unfolded.size(0), input.size(1), 
      int(sqrt(input_unfolded.size(1))), int(sqrt(input_unfolded.size(1))))
   return output

#### Define the neural network architecture

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, backend, shots):
        super(Net, self).__init__()
        
        # Initialize backend and shots attributes
        self.backend = backend
        self.shots = shots

        # Define the convolutional neural network layers
        self.cnn = torch.nn.Sequential(
            ConvBlock(1, 256),
            ConvBlock(256, 128),
            ConvBlock(128, 64),
            torch.nn.Flatten(),
            DenseBlock(64*(IMG_SIZE[0]//8)**2, 128),
            DenseBlock(128, 64),
            torch.nn.Linear(64, 4)
        )

    def forward(self, x):
        x = quantum_convolution(x) # Apply quantum convolution
        x = self.cnn(x) # Apply classical CNN
        return x

## Perform training and evaluation

#### Define the functions to perform training and validation

In [ ]:
def train_one_epoch(model, data_loader, loss_fn, optimizer):
    """Performs one training epoch."""   
    model.train() # Set the model to training mode
    start_time = time.time()

    for batch_index, (inputs, labels) in enumerate(data_loader):
        optimizer.zero_grad() # Clear previous gradients
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward() # Backpropagation
        optimizer.step() # Update model parameters

        _, predicted_labels = torch.max(outputs, 1)
        _, true_labels = torch.max(labels, 1)
        correct_predictions = (predicted_labels == true_labels).sum().item()   
        accuracy = correct_predictions / BATCH_SIZE * 100
    
    end_time = time.time()
    epoch_time = (end_time - start_time)//60
    return loss, accuracy, epoch_time

In [ ]:
def evaluate(model, data_loader, loss_fn):
    """Performs evaluation of the model."""
    model.eval() # Set the model to evaluation mode
    total_loss = 0.0 
    correct_predictions = 0
    start_time = time.time()

    with torch.no_grad():
        for batch_index, (inputs, labels) in enumerate(data_loader):
            outputs = model(inputs)
            total_loss += loss_fn(outputs, labels).item()
            _, predicted_labels = torch.max(outputs, 1)
            _, true_labels = torch.max(labels, 1)
            correct_predictions += (
                predicted_labels == true_labels).sum().item()
    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions / len(data_loader.dataset) * 100
    end_time = time.time()
    evaluation_time = (end_time - start_time)//60
    return avg_loss, accuracy, evaluation_time


#### Perform training and validation

In [ ]:
# Choose the of times the quantum circuits will be executed
SHOTS = 1000

# Choose the backend on which the quantum circuits will be executed
backend = Aer.get_backend('qasm_simulator')

# Instantiate the convolutional neural network
model = Net(backend, SHOTS)

# Choose the loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Choose the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Choose the number of epochs
EPOCHS = 30

# Define the lists to store training and validation metrics
train_losses = []
train_accuracies = []
validation_losses = []
validation_accuracies = []
model_state_dicts = []

# Perform training and validation
for epoch in range(EPOCHS):
    print(f'EPOCH {epoch + 1}:')

    # Perform one training epoch
    train_loss, train_accuracy, train_time = train_one_epoch(
        model, train_loader, loss_fn, optimizer)
    train_losses.append(train_loss.item())
    train_accuracies.append(train_accuracy)

    print('TRAIN: loss {:.3f}; accuracy {:.2f}%; time {:.0f}s'.format(
          train_loss.item(), train_accuracy, train_time))

    # Add model state dict to the list
    model_state_dicts.append(model.state_dict())

    # Perform validation
    validation_loss, validation_accuracy, validation_time = evaluate(
        model, validation_loader, loss_fn) 
    validation_losses.append(validation_loss)
    validation_accuracies.append(validation_accuracy)

    print('VALIDATION: loss {:.3f}; accuracy {:.3f}%; time {:.2f}s'.format(
          validation_loss, validation_accuracy, validation_time))
    
    print()

#### Show the results

In [ ]:
# Show the best accuracy
best_accuracy = max(validation_accuracies)
best_model_idx = validation_accuracies.index(best_accuracy)
print('Best accuracy on validation ds: {:.3}% obtained at the {}th iteration'
      .format(best_accuracy, best_model_idx))

In [ ]:
# Show the training convergence
# Merge plots into one plot with two subplots
plt.figure(figsize=(6, 6))

# First subplot for training metrics
plt.subplot(2, 1, 1)
plt.plot(train_losses, label='Loss', color='blue')
plt.plot(np.array(train_accuracies)/100, label='Accuracy', color='orange')
plt.ylabel('Training Metrics')
plt.legend()
plt.grid(True)

# Second subplot for validation metrics
plt.subplot(2, 1, 2)
plt.plot(validation_losses, label='Loss', color='blue')
plt.plot(np.array(validation_accuracies)/100, label='Accuracy', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Validation Metrics')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

#### Save the best model

In [ ]:
best_model = model_state_dicts[best_model_idx]
torch.save(best_model, r'C:\Users\giuseppe.dambruoso\OneDrive - LUTECH SPA\Desktop\Progetto\BestModel')